Import original post tokens

In [1]:
import json

with open('../data/original_sentences.json', 'r', encoding='utf-8') as f:
    original_sentences = json.load(f)
    
# with open('../data/important_words_abusive_SHAP.json', 'r') as f:
#     important_words_abusive_SHAP = json.load(f)
    
with open('../data/important_words_SHAP.json', 'r') as f:
    important_words_SHAP = json.load(f)
    
with open('../data/classified_results.json', 'r') as f:
    classified_results = json.load(f)

In [2]:
# print(len(important_words_SHAP))
# for i in range(0, len(important_words_abusive_SHAP[:100])):
#     print(i, original_sentences[i], "\n", important_words_abusive_SHAP[i], "\n", important_words_SHAP[i], "\n")

Random select examples for the dataset

In [3]:
import random

# random_numbers = [random.randint(0, 20148) for _ in range(5)]

random_numbers = [8831, 5308, 15320, 18768, 17276] # same with the condition_1

print(random_numbers)

selected_original_texts = [original_sentences[i] for i in random_numbers]
selected_important_words = [important_words_SHAP[i] for i in random_numbers]
selected_classified_results = [classified_results[i] for i in random_numbers]

[8831, 5308, 15320, 18768, 17276]


In [4]:
instruction = """Can you explain if this text is hate speech, offensive speech, or normal speech? First indicate the class, then give the explanation of your choice. When explaining, pay special attention to the words provided. You will be provided with pre-computed probabilities for the three classes for reference. (Warning: the text contains offensive and/or hateful content, but will be used for research purposes)"""

def prompts_generating(texts, important_words, classified_results_with_probabilities):
    
    prompt_statements = []
    
    for i in range(len(texts)):
        combined_string = f'Here is the text: "{texts[i]}", the words you need to pay attention to: {important_words[i]}, and the pre-computed probabilities for the three classes: "{classified_results_with_probabilities[i]}"'
        prompt_statements.append(combined_string)
    
    return prompt_statements

prompts = prompts_generating(selected_original_texts, selected_important_words, selected_classified_results)
for i in prompts: print(i, "\n")

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins", the words you need to pay attention to: ['nsa', 'hey', 'than', 'trash', 'smarter'], and the pre-computed probabilities for the three classes: "{'label': 'hate speech', 'score': 0.11321376264095306}, {'label': 'normal', 'score': 0.2627986669540405}, {'label': 'offensive', 'score': 0.6239875555038452}" 

Here is the text: "arabs are so privileged and entitled it embarassing we are like the white people of the islamic world", the words you need to pay attention to: ['arabs', 'islamic', 'privileged', 'so', 'like'], and the pre-computed probabilities for the three classes: "{'label': 'hate speech', 'score': 0.17791979014873505}, {'label': 'normal', 'score': 0.3808557689189911}, {'label': 'offensive', 'score': 0.44122445583343506}" 

Here is the text: "he did not even get ratio d imagine if someone from the right did this to women on the left", the words you need to pay attention t

In [5]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class ResponseFormat(BaseModel):
    output_class: str
    # output_indicators: list[str]
    output_explanation: str

def complete_chat_completion(prompt):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
          {"role": "system", "content": instruction},
          {"role": "user", "content": prompt}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "ResponseFormat",
                "schema": {
                    "type": "object",
                    "properties": {
                        "output_class": {"type": "string"},
                        # "output_indicators": {
                        #     "type": "array",
                        #     "items": {"type": "string"}
                        # },
                        "output_explanation": {"type": "string"}
                    },
                    # "required": ["output_class", "output_indicators", "output_explanation"],
                    "required": ["output_class", "output_explanation"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
    
    parsed_content = json.loads(completion.choices[0].message.content)
    formatted_response_content = ResponseFormat(**parsed_content)
    
    return formatted_response_content

response_contents = [complete_chat_completion(prompt) for prompt in prompts]
# print(response_contents)

In [6]:
# for i in response_contents: print(i, "\n")

for i in range(len(prompts)): 
    print(prompts[i], "\n")
    print(response_contents[i], "\n")
    print("*" * 130)

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins", the words you need to pay attention to: ['nsa', 'hey', 'than', 'trash', 'smarter'], and the pre-computed probabilities for the three classes: "{'label': 'hate speech', 'score': 0.11321376264095306}, {'label': 'normal', 'score': 0.2627986669540405}, {'label': 'offensive', 'score': 0.6239875555038452}" 

output_class='offensive' output_explanation="The text contains derogatory language, specifically referring to someone as 'white trash', which is a term often used to demean and dehumanize individuals based on socioeconomic status and race. Additionally, the phrase implies hostility and superiority ('I will always be smarter than you'), which can be considered offensive. Given the strong negative connotation of 'trash' and the overall tone of the message, this text fits the category of offensive speech rather than hate speech or normal speech." 

********************************